In [1]:
%pip install -q huggingface_hub datasets python-dotenv openai-whisper tqdm ipywidgets soundfile librosa


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from dotenv import load_dotenv
from huggingface_hub import notebook_login
load_dotenv()
notebook_login()

True

notebook_login()

In [1]:
from datasets import load_dataset

ds = load_dataset("audiofolder", data_dir="tmp/commercial_dataset", split="train")

Resolving data files:   0%|          | 0/1983 [00:00<?, ?it/s]

In [5]:
ds = ds.train_test_split(test_size=0.1, seed=42)
ds

DatasetDict({
    train: Dataset({
        features: ['audio', 'shasum', 'title', 'text', 'original_filename'],
        num_rows: 1783
    })
    test: Dataset({
        features: ['audio', 'shasum', 'title', 'text', 'original_filename'],
        num_rows: 199
    })
})

In [3]:
minds = load_dataset("PolyAI/minds14", name="en-US", split="train")
minds = minds.train_test_split(test_size=0.2)

In [4]:
minds

DatasetDict({
    train: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 450
    })
    test: Dataset({
        features: ['path', 'audio', 'transcription', 'english_transcription', 'intent_class', 'lang_id'],
        num_rows: 113
    })
})

In [4]:
ds.push_to_hub("rimraf/commercial_dataset")

Uploading the dataset shards:   0%|          | 0/64 [00:00<?, ?it/s]

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [2]:
import whisper
model = whisper.load_model("medium")
t = model.transcribe("tmp/commercial_dataset/data/_1dO4z1Wc-Q.wav")
t['text']

/Users/ryan/Code/killallgit/crawler-py/.venv/lib/python3.11/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, 

" I like it. Yep, it's mine. The prices blow my mind I feel so rich. Oh, yeah. I feel like a billionaire Download the team you up and shot like a billionaire You"

In [ ]:
import os
import json
import whisper
import multiprocessing
from tqdm import tqdm
from typing import List, Optional
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def transcribe_audio(file_path: str, model: whisper.Whisper) -> Optional[str]:
    """
    Transcribe a single audio file
    
    :param file_path: Path to audio file
    :param model: Whisper model
    :return: Transcription text or None
    """
    try:
        # Validate file existence
        if not os.path.exists(file_path):
            logger.warning(f"File not found: {file_path}")
            return None
        
        # Simple transcription
        result = model.transcribe(file_path)
        return result['text'].strip()
    
    except Exception as e:
        logger.error(f"Transcription error for {file_path}: {e}")
        return None

def parallel_transcribe(
    model: whisper.Whisper, 
    file_paths: List[str],
    num_workers: Optional[int] = None
) -> List[Optional[str]]:
    """
    Parallel transcription of multiple files
    
    :param model: Whisper model
    :param file_paths: List of file paths to transcribe
    :param num_workers: Number of parallel workers
    :return: List of transcriptions
    """
    if num_workers is None:
        num_workers = max(1, multiprocessing.cpu_count() - 2)
    
    with multiprocessing.Pool(num_workers) as pool:

        def transcribe_func(path):
            return transcribe_audio(path, model)
        
        transcriptions = list(tqdm(
            pool.imap(transcribe_func, file_paths), 
            total=len(file_paths)
        ))
    
    return transcriptions

def process_dataset(
    metadata_path: str = 'tmp/commercial_dataset/metadata.jsonl',
    data_dir: str = 'tmp/commercial_dataset/data',
    model_size: str = 'medium'
) -> None:
    """
    Main function to transcribe a dataset
    
    :param metadata_path: Path to metadata file
    :param data_dir: Directory with audio files
    :param model_size: Whisper model size
    """
    try:
        # 1. Load Whisper model
        logger.info(f"Loading Whisper {model_size} model")
        model = whisper.load_model(model_size)
        
        # 2. Load metadata
        with open(metadata_path, 'r', encoding='utf-8') as f:
            metadata = [json.loads(line) for line in f]
        
        # 3. Filter files to transcribe
        to_transcribe = [
            entry for entry in metadata 
            if not entry.get('text')
        ]
        logger.info(f"Found {len(to_transcribe)} files to transcribe")
        
        # 4. Prepare full file paths
        full_paths = [
            os.path.join(data_dir, entry['file_name']) 
            for entry in to_transcribe
        ]
        
        # 5. Parallel transcription
        transcriptions = parallel_transcribe(model, full_paths)
        
        # 6. Update metadata
        for entry, transcription in zip(to_transcribe, transcriptions):
            if transcription:
                entry['transcription'] = transcription
        
        # 7. Save updated metadata
        with open(metadata_path, 'w', encoding='utf-8') as f:
            for entry in metadata:
                f.write(json.dumps(entry) + '\n')
        
        logger.info("Transcription process completed successfully")
    
    except Exception as e:
        logger.error(f"Transcription failed: {e}")
        raise

# Execution methods
def main():
    process_dataset()

# Ensure this is the primary way of running the script
if __name__ == '__main__':
    main()

# Alternative ways to import and use
def run_transcription(
    metadata_path: str, 
    data_dir: str, 
    model_size: str = 'medium'
):
    """
    External function to trigger transcription
    """
    process_dataset(metadata_path, data_dir, model_size)


2024-11-10 08:37:39,241 - INFO - Loading Whisper medium model
2024-11-10 08:37:44,809 - INFO - Found 2009 files to transcribe
  0%|          | 0/2009 [00:00<?, ?it/s]
2024-11-10 08:37:45,618 - ERROR - Transcription failed: Can't pickle local object 'parallel_transcribe.<locals>.<lambda>'


AttributeError: Can't pickle local object 'parallel_transcribe.<locals>.<lambda>'